In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import splitfolders

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/insects data'

In [ ]:
splitfolders.ratio(data_path, 'insects data splited', ratio=(0.8, 0.15, 0.05), seed=42, group_prefix=None)

Copying files: 3746 files [00:20, 183.47 files/s]


In [ ]:
train = ImageDataGenerator(rescale=1/255)
val = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_data = train.flow_from_directory('/content/insects data splited/train', target_size=(48,48), 
                          class_mode='categorical', batch_size = 30, shuffle=True)
val_data = val.flow_from_directory('/content/insects data splited/val', target_size=(48,48),
                                  class_mode='categorical', batch_size=30, shuffle=True)
test_data = test.flow_from_directory('/content/insects data splited/test', target_size=(48,48), 
                          class_mode='categorical', batch_size = 30, shuffle=True)

Found 3587 images belonging to 5 classes.
Found 1037 images belonging to 5 classes.
Found 370 images belonging to 5 classes.


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(48, 48,3)),
    layers.experimental.preprocessing.RandomRotation(0.05),
    layers.experimental.preprocessing.RandomContrast(0.23),
    layers.experimental.preprocessing.RandomZoom(0.2)
  ]
)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(48, 48, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [ ]:
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
epochs = 50
optimizer = 'adam'

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
callbacks = [ModelCheckpoint('/content/drive/MyDrive/saved models/insects-data-{epoch:02d}-{val_accuracy:.4f}.hdf5')]

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=epochs, batch_size=30, callbacks = callbacks)

Epoch 1/50
120/120 [==============================] - 101s 813ms/step - loss: 1.4693 - accuracy: 0.4382 - val_loss: 3.2260 - val_accuracy: 0.2382
Epoch 2/50
120/120 [==============================] - 99s 824ms/step - loss: 1.1809 - accuracy: 0.5456 - val_loss: 2.5641 - val_accuracy: 0.2758
Epoch 3/50
120/120 [==============================] - 99s 822ms/step - loss: 1.0854 - accuracy: 0.5793 - val_loss: 1.9939 - val_accuracy: 0.2546
Epoch 4/50
120/120 [==============================] - 96s 804ms/step - loss: 0.9823 - accuracy: 0.6273 - val_loss: 1.7850 - val_accuracy: 0.4050
Epoch 5/50
120/120 [==============================] - 100s 829ms/step - loss: 0.9090 - accuracy: 0.6512 - val_loss: 0.9566 - val_accuracy: 0.6249
Epoch 6/50
120/120 [==============================] - 99s 813ms/step - loss: 0.8292 - accuracy: 0.6847 - val_loss: 0.7993 - val_accuracy: 0.6808
Epoch 7/50
120/120 [==============================] - 100s 835ms/step - loss: 0.7570 - accuracy: 0.7117 - val_loss: 0.6852 - val

In [ ]:
scores = model.evaluate(test_data, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 93.78%


In [ ]:
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])